# Requirements
For this tutorial we are going to use DynaSig-ML python package. The instructions for installation are available in: https://dynasigml.readthedocs.io/en/latest/install_guide.html

Also NRGten package is also required. Instructions for instalation are available at:


The following python libraries are also required:

-'dynasigml'

-'nrgten'

-'json'

-'scikit-learn'

-'matplotlib'

-'scipy'

In [1]:
pip install nrgten dynasigml scikit-learn matplotlib scipy

Note: you may need to restart the kernel to use updated packages.


### Adapting NRGten to work with FlexAID atomtypes
It is necessary to change the two config files "amino_acids.atomtypes" and "amino_acids.masses" by the ones provided in this directory. The following code does it automatially:

In [2]:
import sys
import importlib.util
import os
import shutil

library_name = 'nrgten' 

spec = importlib.util.find_spec(library_name) #find the path for the NRGten lib

if spec:
    print("Path of the library:", spec.origin)
    base_folder = os.path.dirname(spec.origin)
    old_dir = os.path.join(base_folder, 'config', 'old').replace('\\', '/')
    if not os.path.isdir(old_dir):
        os.mkdir(old_dir)
        os.rename(os.path.join(base_folder, 'config', 'amino_acids.atomtypes'), os.path.join(old_dir, 'amino_acids.atomtypes'))
        os.rename(os.path.join(base_folder, 'config', 'amino_acids.masses'), os.path.join(old_dir, 'amino_acids.masses'))
    
        shutil.copy('amino_acids.atomtypes', os.path.join(base_folder, 'config', 'amino_acids.atomtypes'))
        shutil.copy('amino_acids.masses', os.path.join(base_folder, 'config', 'amino_acids.masses'))
else:
    print("Library not found")

Path of the library: /opt/anaconda3/lib/python3.8/site-packages/nrgten/__init__.py


### Importing all libraries

In [4]:
from dynasigml.dynasig_df import DynaSigDF
from dynasigml.dynasig_df import load_pickled_dynasig_df
from dynasigml.dynasig_ml_model import DynaSigML_Model
import numpy as np
import json
from sklearn.metrics import r2_score, roc_auc_score
from nrgten.encom import ENCoM
from scipy import stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd

### Preparing files for DynaSig-ML

After running FlexAID we selected the poses with the top 10 CFs for Mu opioid receptors.

The CF value of each pose is written in the file 'mu_data.txt' and the poses of each are registered in the directory './mu_poses/'.

The values of Emax were obtained from CHEMBl library and are also listed in 'mu_data.txt'. The CHEMBL code for Mu is CHEMBL233.

The tables with all values of Emax are listed if the files: './mu_data.txt' and './kappa_data.txt'.
For this first step we are going to construct three lists:
   * file_names:
           This list contains the path to each pose file
   * exp_data:
           List of Emax and CFs for each pose (needs to be in the same order as file_names)
   * list_of_atypes:
           List of .atomtypes files of all ligands
   * list_of_masses:
           List of .masses files of all ligands, these files specifies the ligand centroid and the name of the mass
   * output_name_file: 
           string with name of the data frame file generated by DynaSigDF
   * exp_labels:
           string with the labels for the values present in the exp_data file
   * betas:
           velues of beta we are going to test

In [5]:
file_names=[]
exp_data=[]
list_of_atypes=[]
list_of_masses=[]
exp_labels=['Emax','CF']
betas=[1]
with open('./mu_data.txt','r') as t1:
    files=t1.readlines()
    for file in files:
        file_names.append('./mu_files/{}'.format(file.split()[0]))
        exp_data.append([file.split()[1]])
        list_of_atypes.append('./mu_files/{}_flexaid.atomtypes'.format(file.split()[0][:-4]))
        list_of_masses.append('./mu_files/{}_flexaid.masses'.format(file.split()[0][:-4]))

### Reading the FlexAID matrix

This function opens the contact matrix and invert the contact interaction values

In [6]:
def flex_aid_matrix():
    matrix=np.array([np.zeros(41)]*41)

    with open("FlexAID.dat","r") as t1:
        texto=t1.readlines()
        for line in texto:
            matrix[int(line.split("=")[0].split('-')[0])][int(line.split("=")[0].split('-')[1])]=-float(line.split("=")[1])
    return(matrix)

### Running DynasigML

We import the function DynaSigDF to create a data frame called 'mu_dsdf' that will contain the dynamical signature and the experimental values of Emax. This data frame will then be used to train LASSO model. <span style="color:red">THIS MAY TAKE 1-2 HOURs IN A COMMON LAPTOP</span> 

In [7]:
def encom_flexaid(filename ,added_atypes='', added_massdef=''):
    print(filename)
    matrix=flex_aid_matrix()
    return ENCoM(filename, interact_mat=matrix, added_atypes=added_atypes, added_massdef=added_massdef)

output_name_file='./dsdf/mu_flexaid_dsdf'
exp_labels=['Emax']
betas=[np.e**(-0.5)]

dsdf = DynaSigDF(file_names, exp_data, exp_labels, output_name_file, beta_values=betas,added_atypes_list=list_of_atypes, added_massdef_list=list_of_masses, models=[encom_flexaid], models_labels=["ENCoM_flexaid"])


./mu_files/CHEMBL4074110_2_0.pdb
./mu_files/CHEMBL4074110_6_0.pdb
./mu_files/CHEMBL4074110_4_0.pdb
./mu_files/CHEMBL4074110_5_0.pdb
./mu_files/CHEMBL4074110_3_1.pdb
./mu_files/CHEMBL4074110_0_0.pdb
./mu_files/CHEMBL4074110_6_1.pdb
./mu_files/CHEMBL4074110_7_1.pdb
./mu_files/CHEMBL4074110_0_1.pdb
./mu_files/CHEMBL4074110_2_2.pdb
./mu_files/CHEMBL566749_8_0.pdb
./mu_files/CHEMBL566749_8_1.pdb
./mu_files/CHEMBL566749_5_1.pdb
./mu_files/CHEMBL566749_1_1.pdb
./mu_files/CHEMBL566749_5_2.pdb
./mu_files/CHEMBL566749_9_1.pdb
./mu_files/CHEMBL566749_9_2.pdb
./mu_files/CHEMBL566749_6_1.pdb
./mu_files/CHEMBL566749_3_1.pdb
./mu_files/CHEMBL566749_8_2.pdb
./mu_files/CHEMBL481547_1_0.pdb
./mu_files/CHEMBL481547_9_0.pdb
./mu_files/CHEMBL481547_2_0.pdb
./mu_files/CHEMBL481547_6_0.pdb
./mu_files/CHEMBL481547_1_1.pdb
./mu_files/CHEMBL481547_6_1.pdb
./mu_files/CHEMBL481547_7_1.pdb
./mu_files/CHEMBL481547_3_0.pdb
./mu_files/CHEMBL481547_5_0.pdb
./mu_files/CHEMBL481547_3_1.pdb
./mu_files/CHEMBL1650843_9_0.p

./mu_files/CHEMBL364844_6_0.pdb
./mu_files/CHEMBL364844_2_0.pdb
./mu_files/CHEMBL364844_5_0.pdb
./mu_files/CHEMBL364844_1_0.pdb
./mu_files/CHEMBL364844_0_0.pdb
./mu_files/CHEMBL364844_2_1.pdb
./mu_files/CHEMBL364844_5_1.pdb
./mu_files/CHEMBL364844_7_0.pdb
./mu_files/CHEMBL259658_2_0.pdb
./mu_files/CHEMBL259658_4_0.pdb
./mu_files/CHEMBL259658_9_0.pdb
./mu_files/CHEMBL259658_8_0.pdb
./mu_files/CHEMBL259658_5_0.pdb
./mu_files/CHEMBL259658_4_1.pdb
./mu_files/CHEMBL259658_3_0.pdb
./mu_files/CHEMBL259658_1_0.pdb
./mu_files/CHEMBL259658_2_1.pdb
./mu_files/CHEMBL259658_4_2.pdb
./mu_files/CHEMBL190785_6_0.pdb
./mu_files/CHEMBL190785_6_1.pdb
./mu_files/CHEMBL190785_4_0.pdb
./mu_files/CHEMBL190785_6_2.pdb
./mu_files/CHEMBL190785_9_0.pdb
./mu_files/CHEMBL190785_4_1.pdb
./mu_files/CHEMBL190785_4_2.pdb
./mu_files/CHEMBL190785_4_3.pdb
./mu_files/CHEMBL190785_6_3.pdb
./mu_files/CHEMBL190785_6_4.pdb
./mu_files/CHEMBL4205200_7_0.pdb
./mu_files/CHEMBL4205200_5_0.pdb
./mu_files/CHEMBL4205200_1_1.pdb
./mu_

./mu_files/CHEMBL4210647_5_1.pdb
./mu_files/CHEMBL4210647_0_1.pdb
./mu_files/CHEMBL4210647_7_1.pdb
./mu_files/CHEMBL4210647_2_1.pdb
./mu_files/CHEMBL259299_5_0.pdb
./mu_files/CHEMBL259299_0_0.pdb
./mu_files/CHEMBL259299_0_1.pdb
./mu_files/CHEMBL259299_8_0.pdb
./mu_files/CHEMBL259299_1_0.pdb
./mu_files/CHEMBL259299_0_2.pdb
./mu_files/CHEMBL259299_0_3.pdb
./mu_files/CHEMBL259299_6_4.pdb
./mu_files/CHEMBL259299_1_1.pdb
./mu_files/CHEMBL259299_1_2.pdb
./mu_files/CHEMBL4215165_7_0.pdb
./mu_files/CHEMBL4215165_7_1.pdb
./mu_files/CHEMBL4215165_3_0.pdb
./mu_files/CHEMBL4215165_1_0.pdb
./mu_files/CHEMBL4215165_9_0.pdb
./mu_files/CHEMBL4215165_8_0.pdb
./mu_files/CHEMBL4215165_3_1.pdb
./mu_files/CHEMBL4215165_0_0.pdb
./mu_files/CHEMBL4215165_1_1.pdb
./mu_files/CHEMBL4215165_3_2.pdb
./mu_files/CHEMBL1852555_0_0.pdb
./mu_files/CHEMBL1852555_4_0.pdb
./mu_files/CHEMBL1852555_5_0.pdb
./mu_files/CHEMBL1852555_1_0.pdb
./mu_files/CHEMBL1852555_3_0.pdb
./mu_files/CHEMBL1852555_8_0.pdb
./mu_files/CHEMBL185

./mu_files/CHEMBL4212478_5_0.pdb
./mu_files/CHEMBL4212478_8_0.pdb
./mu_files/CHEMBL4212478_6_0.pdb
./mu_files/CHEMBL4212478_2_0.pdb
./mu_files/CHEMBL4212478_0_0.pdb
./mu_files/CHEMBL4212478_3_0.pdb
./mu_files/CHEMBL4212478_9_0.pdb
./mu_files/CHEMBL4212478_7_0.pdb
./mu_files/CHEMBL4212478_5_2.pdb
./mu_files/CHEMBL4212478_0_1.pdb
./mu_files/CHEMBL3787016_1_0.pdb
./mu_files/CHEMBL3787016_8_0.pdb
./mu_files/CHEMBL3787016_3_0.pdb
./mu_files/CHEMBL3787016_7_0.pdb
./mu_files/CHEMBL3787016_4_0.pdb
./mu_files/CHEMBL3787016_6_0.pdb
./mu_files/CHEMBL3787016_5_0.pdb
./mu_files/CHEMBL3787016_4_1.pdb
./mu_files/CHEMBL3787016_8_1.pdb
./mu_files/CHEMBL3787016_7_1.pdb
./mu_files/CHEMBL519100_9_0.pdb
./mu_files/CHEMBL519100_9_1.pdb
./mu_files/CHEMBL519100_1_0.pdb
./mu_files/CHEMBL519100_9_2.pdb
./mu_files/CHEMBL519100_1_1.pdb
./mu_files/CHEMBL519100_1_2.pdb
./mu_files/CHEMBL519100_1_3.pdb
./mu_files/CHEMBL519100_5_2.pdb
./mu_files/CHEMBL519100_1_4.pdb
./mu_files/CHEMBL519100_5_3.pdb
./mu_files/CHEMBL406

./mu_files/CHEMBL1852385_9_0.pdb
./mu_files/CHEMBL1852385_2_0.pdb
./mu_files/CHEMBL1852385_6_0.pdb
./mu_files/CHEMBL1852385_1_0.pdb
./mu_files/CHEMBL1852385_8_0.pdb
./mu_files/CHEMBL1852385_4_0.pdb
./mu_files/CHEMBL1852385_9_1.pdb
./mu_files/CHEMBL1852385_6_1.pdb
./mu_files/CHEMBL1852385_1_1.pdb
./mu_files/CHEMBL2397015_1_0.pdb
./mu_files/CHEMBL2397015_0_0.pdb
./mu_files/CHEMBL2397015_6_0.pdb
./mu_files/CHEMBL2397015_4_0.pdb
./mu_files/CHEMBL2397015_3_0.pdb
./mu_files/CHEMBL2397015_4_1.pdb
./mu_files/CHEMBL2397015_8_0.pdb
./mu_files/CHEMBL2397015_5_0.pdb
./mu_files/CHEMBL2397015_2_0.pdb
./mu_files/CHEMBL2397015_9_0.pdb
./mu_files/CHEMBL481707_3_0.pdb
./mu_files/CHEMBL481707_0_0.pdb
./mu_files/CHEMBL481707_1_0.pdb
./mu_files/CHEMBL481707_6_0.pdb
./mu_files/CHEMBL481707_2_0.pdb
./mu_files/CHEMBL481707_4_0.pdb
./mu_files/CHEMBL481707_6_1.pdb
./mu_files/CHEMBL481707_3_2.pdb
./mu_files/CHEMBL481707_9_0.pdb
./mu_files/CHEMBL481707_5_1.pdb
./mu_files/CHEMBL2178338_6_0.pdb
./mu_files/CHEMBL217

./mu_files/CHEMBL4219000_8_1.pdb
./mu_files/CHEMBL4219000_2_1.pdb
./mu_files/CHEMBL4219000_9_1.pdb
./mu_files/CHEMBL4219000_5_1.pdb
./mu_files/CHEMBL4219000_6_1.pdb
./mu_files/CHEMBL4219000_9_2.pdb
./mu_files/CHEMBL1852672_7_0.pdb
./mu_files/CHEMBL1852672_1_0.pdb
./mu_files/CHEMBL1852672_8_0.pdb
./mu_files/CHEMBL1852672_0_0.pdb
./mu_files/CHEMBL1852672_7_1.pdb
./mu_files/CHEMBL1852672_0_1.pdb
./mu_files/CHEMBL1852672_8_1.pdb
./mu_files/CHEMBL1852672_0_2.pdb
./mu_files/CHEMBL1852672_0_3.pdb
./mu_files/CHEMBL1852672_1_1.pdb
./mu_files/CHEMBL184646_8_0.pdb
./mu_files/CHEMBL184646_6_0.pdb
./mu_files/CHEMBL184646_6_1.pdb
./mu_files/CHEMBL184646_5_0.pdb
./mu_files/CHEMBL184646_7_0.pdb
./mu_files/CHEMBL184646_8_2.pdb
./mu_files/CHEMBL184646_7_1.pdb
./mu_files/CHEMBL184646_8_3.pdb
./mu_files/CHEMBL184646_7_2.pdb
./mu_files/CHEMBL184646_5_1.pdb
./mu_files/CHEMBL4211156_9_0.pdb
./mu_files/CHEMBL4211156_8_0.pdb
./mu_files/CHEMBL4211156_6_0.pdb
./mu_files/CHEMBL4211156_4_1.pdb
./mu_files/CHEMBL421

./mu_files/CHEMBL1852458_3_1.pdb
./mu_files/CHEMBL1852458_1_0.pdb
./mu_files/CHEMBL1852458_8_3.pdb
./mu_files/CHEMBL1852458_3_2.pdb
./mu_files/CHEMBL1852458_7_0.pdb
./mu_files/CHEMBL4209504_7_0.pdb
./mu_files/CHEMBL4209504_4_0.pdb
./mu_files/CHEMBL4209504_0_0.pdb
./mu_files/CHEMBL4209504_2_0.pdb
./mu_files/CHEMBL4209504_8_0.pdb
./mu_files/CHEMBL4209504_3_0.pdb
./mu_files/CHEMBL4209504_9_0.pdb
./mu_files/CHEMBL4209504_6_0.pdb
./mu_files/CHEMBL4209504_5_0.pdb
./mu_files/CHEMBL4209504_2_1.pdb
./mu_files/CHEMBL4216570_0_0.pdb
./mu_files/CHEMBL4216570_5_0.pdb
./mu_files/CHEMBL4216570_7_0.pdb
./mu_files/CHEMBL4216570_1_0.pdb
./mu_files/CHEMBL4216570_0_1.pdb
./mu_files/CHEMBL4216570_4_0.pdb
./mu_files/CHEMBL4216570_6_0.pdb
./mu_files/CHEMBL4216570_5_1.pdb
./mu_files/CHEMBL4216570_4_1.pdb
./mu_files/CHEMBL4216570_4_2.pdb
./mu_files/CHEMBL1650841_2_0.pdb
./mu_files/CHEMBL1650841_4_0.pdb
./mu_files/CHEMBL1650841_3_0.pdb
./mu_files/CHEMBL1650841_0_0.pdb
./mu_files/CHEMBL1650841_7_0.pdb
./mu_files

./mu_files/CHEMBL184328_5_1.pdb
./mu_files/CHEMBL184328_9_1.pdb
./mu_files/CHEMBL184328_1_2.pdb
./mu_files/CHEMBL4217435_6_0.pdb
./mu_files/CHEMBL4217435_1_0.pdb
./mu_files/CHEMBL4217435_9_0.pdb
./mu_files/CHEMBL4217435_1_1.pdb
./mu_files/CHEMBL4217435_8_0.pdb
./mu_files/CHEMBL4217435_6_1.pdb
./mu_files/CHEMBL4217435_1_2.pdb
./mu_files/CHEMBL4217435_0_0.pdb
./mu_files/CHEMBL4217435_7_0.pdb
./mu_files/CHEMBL4217435_8_1.pdb
./mu_files/CHEMBL461258_2_0.pdb
./mu_files/CHEMBL461258_7_0.pdb
./mu_files/CHEMBL461258_8_0.pdb
./mu_files/CHEMBL461258_7_1.pdb
./mu_files/CHEMBL461258_9_0.pdb
./mu_files/CHEMBL461258_0_0.pdb
./mu_files/CHEMBL461258_3_0.pdb
./mu_files/CHEMBL461258_9_1.pdb
./mu_files/CHEMBL461258_1_0.pdb
./mu_files/CHEMBL461258_0_1.pdb


### Running Leave-one-ligand out performance test

At the moment of this publication the leave_one_out test is not implemented inside DynaSigML. We are going to define some functions that are going to construct our training set and our test set for every ligand. In this validation we are going to use all 10 poses of one ligand as a testing set and use all other poses of the other ligands as a training test and repete this analysis to each ligand.
The functions we are going to define are:

In [12]:
def get_list_10(ligand_name): #gets the id for the 10 poses of each ligand
    list_10 = []
    for name in dsdf.get_file_ids():
        if ligand_name+'_' in name:
            list_10.append(name)
    assert len(list_10) == 10
    return list_10

def get_all_ligands(file_ids): #gets the list of all ligands
    ligands = set()
    for ligand in file_ids:
        lig = ligand.split('_')[0]
        ligands.add(lig)
    return list(ligands)

def divide_ligands(file_ids): #creates the data set of poses
    all_lig=get_all_ligands(file_ids)
    groups=[]
    for lig in all_lig:
        groups.append(get_list_10(lig))
    return groups

def run_all_groups(groups, dsdf): #runs LASSO leave-one-out for all ligands
    coefficients=dict()
    for i in range(len(groups)):
        lasso_data_dict, coefficients[i] = run_group_loo(groups[i], dsdf)
        with open('leave-one-out-mu/lasso_{}.json'.format(i), 'w') as f:
            json.dump(lasso_data_dict, f)
        print("lig "+ str(i+1) + " done!")
        
def std_test_data(dsml_obj, beta, test_data): #standardize the data per position
    standardization = dsml_obj.standardization_dict[beta]
    assert len(standardization) == len(test_data[0, 2:])
    for i in range(2, test_data.shape[1]):
        mean, sd = standardization[i - 2]
        test_data[:, i] -= mean
        if sd > 0:
            test_data[:, i] /= sd
    return test_data

def run_group_loo(group, dsdf): #runs LASSO leave-one-out for one ligand
    dsml_group = DynaSigML_Model('./dsdf/mu_flexaid_dsdf.pickle', test_ids=group, alphas=[2**(3/4)])
    dsml_group.train_test_lasso()
    lasso_dict = dsml_group.lasso_stats_dict
    lasso_data_dict = dict()
    coefficients=dict()
    for beta in lasso_dict:
        coefficients[beta]=dict()
        test_data = std_test_data(dsml_group, beta, dsdf.get_data_array(group, beta=beta))
        lasso_data_dict[beta] = dict()
        for alpha in lasso_dict[beta]:
            trained_lasso_model = lasso_dict[beta][alpha]['model']
            coefficients[beta][alpha]= list(lasso_dict[beta][alpha]['model'].coef_)
            real_values = test_data[:, 1]
            predicted_values = trained_lasso_model.predict(test_data[:, 2:])
            lasso_data_dict[beta][alpha] = [[x for x in real_values], [x for x in predicted_values]]
    return lasso_data_dict, coefficients



### Main

In [13]:
dsdf=load_pickled_dynasig_df("./dsdf/mu_flexaid_dsdf.pickle")
file_ids=dsdf.get_file_ids()
groups=divide_ligands(file_ids)
alphas=[2**(3/4)]
coefficients=run_all_groups(groups, dsdf)

lig 1 done!
lig 2 done!
lig 3 done!
lig 4 done!
lig 5 done!
lig 6 done!
lig 7 done!
lig 8 done!
lig 9 done!
lig 10 done!
lig 11 done!
lig 12 done!
lig 13 done!
lig 14 done!
lig 15 done!
lig 16 done!
lig 17 done!
lig 18 done!
lig 19 done!
lig 20 done!
lig 21 done!
lig 22 done!
lig 23 done!
lig 24 done!
lig 25 done!
lig 26 done!
lig 27 done!
lig 28 done!
lig 29 done!
lig 30 done!
lig 31 done!
lig 32 done!
lig 33 done!
lig 34 done!
lig 35 done!
lig 36 done!
lig 37 done!
lig 38 done!
lig 39 done!
lig 40 done!
lig 41 done!
lig 42 done!
lig 43 done!
lig 44 done!
lig 45 done!
lig 46 done!
lig 47 done!
lig 48 done!
lig 49 done!
lig 50 done!
lig 51 done!
lig 52 done!
lig 53 done!
lig 54 done!
lig 55 done!
lig 56 done!
lig 57 done!
lig 58 done!
lig 59 done!
lig 60 done!
lig 61 done!
lig 62 done!
lig 63 done!
lig 64 done!
lig 65 done!
lig 66 done!
lig 67 done!
lig 68 done!
lig 69 done!
lig 70 done!
lig 71 done!
lig 72 done!
lig 73 done!
lig 74 done!
lig 75 done!
lig 76 done!
lig 77 done!
lig 78 d

### Plotting the data


In [16]:
def get_mean(model_pred,classes):
    n_class=[0]*int(len(model_pred)/10)
    pred_dic=[0]*int(len(model_pred)/10)
    for i in range(int(len(model_pred)/10)):
        n_class[i]=classes[i*10]
        for j in range(10):
            pred_dic[i]+=model_pred[i*10+j]
        pred_dic[i]/=10
    return(pred_dic,n_class)

def analyze_all_groups_loocv(file_ids,betas,alphas):
    all_lasso = dict()
    ligands=list(get_all_ligands(file_ids))
    with open('leave-one-out-mu/lasso_{}.json'.format(0)) as f:
        d = json.load(f)
    betas = [x for x in d]
    alphas = [x for x in d[betas[0]]]
    for beta in betas:
        all_lasso[beta] = dict()
        for alpha in alphas:
            all_lasso[beta][alpha] = [[], []]
    for lig in range(len(ligands)):
        with open('leave-one-out-mu/lasso_{}.json'.format(lig)) as f:
            lasso_dict = json.load(f)
        for beta in betas:
            for alpha in alphas:
                all_lasso[beta][alpha][0] = all_lasso[beta][alpha][0] + lasso_dict[beta][alpha][0]
                all_lasso[beta][alpha][1] = all_lasso[beta][alpha][1] + lasso_dict[beta][alpha][1]   
    for beta in betas:
        for alpha in alphas:
            classes = [x >= 50 for x in all_lasso[beta][alpha][0]]
            pred_dic,classe_n=get_mean(all_lasso[beta][alpha][1],classes)
            lasso_r = pearsonr(classe_n, pred_dic)[0]
            lasso_auc = roc_auc_score(classe_n, pred_dic)
            print("LASSO: beta: {} alpha: {:.2f} pr: {:.2f} auc: {:.2f}".format(np.log(float(beta)), np.log2(float(alpha)), lasso_r, lasso_auc))




In [17]:
analyze_all_groups_loocv(file_ids,betas,alphas)

LASSO: beta: -0.5 alpha: 0.75 pr: 0.46 auc: 0.79
